In [ ]:
# Ganti isi cell yang berisi instalasi & skrip dengan kode berikut

# (Di Jupyter Notebook, gunakan '!' untuk menjalankan pip)
!pip install -q librosa pydub soundfile

import os
import librosa
import numpy as np
import soundfile as sf
import random
import traceback

def load_audio(file_path):
    audio, sr = librosa.load(file_path, sr=None, mono=False)
    # convert to mono if stereo
    if audio.ndim > 1:
        audio = np.mean(audio, axis=0)
    return audio, sr

def save_audio(file_path, audio, sr):
    audio = np.clip(audio, -1.0, 1.0).astype(np.float32)
    sf.write(file_path, audio, sr)

def augment_audio(audio, sr, pitch_factor=0, stretch_factor=1.0):
    y = audio.copy()
    # pitch shift if not zero
    if pitch_factor != 0:
        try:
            y = librosa.effects.pitch_shift(y, sr, n_steps=pitch_factor)
        except Exception:
            # fallback: skip pitch shift
            pass
    # time stretch if not 1.0
    if abs(stretch_factor - 1.0) > 1e-6:
        try:
            # time_stretch requires sufficient length; if error, raise to caller
            y = librosa.effects.time_stretch(y, rate=stretch_factor)
        except Exception:
            raise
    return y

def add_noise(audio, noise_factor=0.005):
    noise = np.random.randn(len(audio))
    audio_noisy = audio + noise_factor * noise
    return np.clip(audio_noisy, -1.0, 1.0)

def augment_dataset(folder_path, target_folder, num_augments=5):
    if not os.path.isdir(folder_path):
        print(f"Folder sumber tidak ditemukan: {folder_path}")
        return

    os.makedirs(target_folder, exist_ok=True)
    files = [f for f in os.listdir(folder_path) if f.lower().endswith('.wav')]

    for file_name in files:
        file_path = os.path.join(folder_path, file_name)
        try:
            audio, sr = load_audio(file_path)
        except Exception as e:
            print(f"Gagal memuat {file_name}: {e}")
            continue

        base, _ = os.path.splitext(file_name)
        for i in range(num_augments):
            try:
                pitch = random.randint(-2, 2)
                stretch = random.uniform(0.9, 1.2)  # lebih konservatif
                # copy original to avoid cumulative transforms
                aug = audio.copy()
                # If audio is very short, pad to ~0.5s to avoid time_stretch errors
                min_len = int(0.5 * sr)
                if len(aug) < min_len:
                    aug = np.pad(aug, (0, min_len - len(aug)), mode='constant')

                try:
                    aug = augment_audio(aug, sr, pitch_factor=pitch, stretch_factor=stretch)
                except Exception:
                    # jika time-stretch gagal, coba hanya pitch shift (or skip)
                    try:
                        aug = librosa.effects.pitch_shift(audio, sr, n_steps=pitch)
                    except Exception:
                        aug = audio.copy()

                aug = add_noise(aug, noise_factor=0.005)

                out_name = f"aug_{i+1}_{base}_p{pitch}_s{stretch:.2f}.wav"
                augmented_file_path = os.path.join(target_folder, out_name)
                save_audio(augmented_file_path, aug, sr)
            except Exception as e:
                print(f"Gagal augmentasi {file_name} iterasi {i+1}: {e}")
                traceback.print_exc()
                continue

        print(f"Augmentasi untuk {file_name} selesai! Hasil disimpan di {target_folder}")

def process_folders():
    suarabuka_folder = 'Suarabuka'
    suaratutup_folder = 'Suaratutup'

    suarabuka_augmented_folder = 'augmentasi_suarabuka'
    suaratutup_augmented_folder = 'augmentasi_suaratutup'

    augment_dataset(suarabuka_folder, suarabuka_augmented_folder)
    augment_dataset(suaratutup_folder, suaratutup_augmented_folder)

if __name__ == '__main__':
    process_folders()

In [9]:
# Ganti isi cell yang berisi instalasi & skrip dengan kode berikut

# (Di Jupyter Notebook, gunakan '!' untuk menjalankan pip)
!pip install -q librosa pydub soundfile


Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import librosa
import numpy as np
import soundfile as sf
import random
import traceback

def load_audio(file_path):
    audio, sr = librosa.load(file_path, sr=None, mono=False)
    # convert to mono if stereo
    if audio.ndim > 1:
        audio = np.mean(audio, axis=0)
    return audio, sr

def save_audio(file_path, audio, sr):
    audio = np.clip(audio, -1.0, 1.0).astype(np.float32)
    sf.write(file_path, audio, sr)

def augment_audio(audio, sr, pitch_factor=0, stretch_factor=1.0):
    y = audio.copy()
    # pitch shift if not zero
    if pitch_factor != 0:
        try:
            y = librosa.effects.pitch_shift(y, sr, n_steps=pitch_factor)
        except Exception:
            # fallback: skip pitch shift
            pass
    # time stretch if not 1.0
    if abs(stretch_factor - 1.0) > 1e-6:
        try:
            # time_stretch requires sufficient length; if error, raise to caller
            y = librosa.effects.time_stretch(y, rate=stretch_factor)
        except Exception:
            raise
    return y

def add_noise(audio, noise_factor=0.005):
    noise = np.random.randn(len(audio))
    audio_noisy = audio + noise_factor * noise
    return np.clip(audio_noisy, -1.0, 1.0)

def augment_dataset(folder_path, target_folder, num_augments=5):
    if not os.path.isdir(folder_path):
        print(f"Folder sumber tidak ditemukan: {folder_path}")
        return

    os.makedirs(target_folder, exist_ok=True)
    files = [f for f in os.listdir(folder_path) if f.lower().endswith('.wav')]

    for file_name in files:
        file_path = os.path.join(folder_path, file_name)
        try:
            audio, sr = load_audio(file_path)
        except Exception as e:
            print(f"Gagal memuat {file_name}: {e}")
            continue

        base, _ = os.path.splitext(file_name)
        for i in range(num_augments):
            try:
                pitch = random.randint(-2, 2)
                stretch = random.uniform(0.9, 1.2)  # lebih konservatif
                # copy original to avoid cumulative transforms
                aug = audio.copy()
                # If audio is very short, pad to ~0.5s to avoid time_stretch errors
                min_len = int(0.5 * sr)
                if len(aug) < min_len:
                    aug = np.pad(aug, (0, min_len - len(aug)), mode='constant')

                try:
                    aug = augment_audio(aug, sr, pitch_factor=pitch, stretch_factor=stretch)
                except Exception:
                    # jika time-stretch gagal, coba hanya pitch shift (or skip)
                    try:
                        aug = librosa.effects.pitch_shift(audio, sr, n_steps=pitch)
                    except Exception:
                        aug = audio.copy()

                aug = add_noise(aug, noise_factor=0.005)

                out_name = f"aug_{i+1}_{base}_p{pitch}_s{stretch:.2f}.wav"
                augmented_file_path = os.path.join(target_folder, out_name)
                save_audio(augmented_file_path, aug, sr)
            except Exception as e:
                print(f"Gagal augmentasi {file_name} iterasi {i+1}: {e}")
                traceback.print_exc()
                continue

        print(f"Augmentasi untuk {file_name} selesai! Hasil disimpan di {target_folder}")

def process_folders():
    suarabuka_folder = 'Suarabuka'
    suaratutup_folder = 'Suaratutup'

    suarabuka_augmented_folder = 'augmentasi_suarabuka'
    suaratutup_augmented_folder = 'augmentasi_suaratutup'

    augment_dataset(suarabuka_folder, suarabuka_augmented_folder)
    augment_dataset(suaratutup_folder, suaratutup_augmented_folder)

if __name__ == '__main__':
    process_folders()

Augmentasi untuk Buka 2.wav selesai! Hasil disimpan di augmentasi_suarabuka
Augmentasi untuk Buka 4.wav selesai! Hasil disimpan di augmentasi_suarabuka
Augmentasi untuk Buka 1.wav selesai! Hasil disimpan di augmentasi_suarabuka
Augmentasi untuk Buka 3.wav selesai! Hasil disimpan di augmentasi_suarabuka
Augmentasi untuk Buka 5.wav selesai! Hasil disimpan di augmentasi_suarabuka
Augmentasi untuk Tutup 4.wav selesai! Hasil disimpan di augmentasi_suaratutup
Augmentasi untuk Tutup 3.wav selesai! Hasil disimpan di augmentasi_suaratutup
Augmentasi untuk Tutup 2.wav selesai! Hasil disimpan di augmentasi_suaratutup
Augmentasi untuk Tutup 1.wav selesai! Hasil disimpan di augmentasi_suaratutup
Augmentasi untuk Tutup 5.wav selesai! Hasil disimpan di augmentasi_suaratutup
